In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# Define a classification chain to categorize queries
classification_chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either `LangChain`, `Anthropic`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOllama(model="llama3.2")
    | StrOutputParser()
)

# Example usage
print(classification_chain.invoke({"question": "How do I call Anthropic?"}))  # Expected: "Anthropic"


In [10]:
# LangChain sub-chain
langchain_chain = (
    PromptTemplate.from_template(
        """You are an expert in LangChain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOllama(model="llama3.2")
)

# Anthropic sub-chain
anthropic_chain = (
    PromptTemplate.from_template(
        """You are an expert in Anthropic. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOllama(model="llama3.2")
)

# General question sub-chain
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOllama(model="llama3.2")
)


In [ ]:
from langchain_core.runnables import RunnableLambda

# Define routing logic
def route(info):
    if "anthropic" in info["topic"].lower():
        return anthropic_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

# Create a pipeline that classifies and routes the query
full_chain = {"topic": classification_chain, "question": lambda x: x["question"]} | RunnableLambda(route)

# Example invocations
print(full_chain.invoke({"question": "How do I use Anthropic?"}))  # Routes to Anthropic chain
print("#############################")
print(full_chain.invoke({"question": "How do I use LangChain?"}))  # Routes to LangChain chain
print("#############################")
print(full_chain.invoke({"question": "What's 2 + 2?"}))  # Routes to General chain


In [ ]:
from langchain_core.runnables import RunnableBranch

# Define the branch-based routing logic
branch = RunnableBranch(
    (lambda x: "anthropic" in x["topic"].lower(), anthropic_chain),
    (lambda x: "langchain" in x["topic"].lower(), langchain_chain),
    general_chain,  # Default chain
)

# Create a pipeline using `RunnableBranch`
full_chain = {"topic": classification_chain, "question": lambda x: x["question"]} | branch

# Example invocations
print(full_chain.invoke({"question": "How do I use Anthropic?"}))  # Routes to Anthropic chain
print("#############################")
print(full_chain.invoke({"question": "How do I use LangChain?"}))  # Routes to LangChain chain
print("#############################")
print(full_chain.invoke({"question": "What's 2 + 2?"}))  # Routes to General chain


In [ ]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama

# Define subject-specific prompts
physics_template = """You are a physics professor. Explain concepts concisely.
Here is a question: {query}"""

math_template = """You are a mathematician. Break down problems into steps.
Here is a question: {query}"""

# Generate embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Define function to choose best-matching prompt and return formatted string
def prompt_router(input):
    if isinstance(input, str):  # Ensure input is a dict
        input = {"query": input}

    query = input["query"]  # Extract query string
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    
    return most_similar.format(query=query)  # Ensure output is a string

# Create chain
chain = (
    RunnableLambda(lambda x: {"query": x})  # Convert raw input into dict
    | RunnableLambda(prompt_router)  # Routes to correct prompt
    | ChatOllama(model="llama3.2")  # Processes query with Ollama
    | StrOutputParser()  # Extracts output as a string
)

# Example usage
print(chain.invoke("What’s a black hole?"))  # Routes to Physics chain
print("#############################")
print(chain.invoke("What’s a path integral?"))  # Routes to Math chain
